In [18]:
import pandas as pd
import spacy

nlp = spacy.load("en_core_web_sm")

def process_query(query_text, df):
    doc = nlp(query_text.lower())
    
    select_cols = []
    from_table = ""
    where_cond = ""
    group_by_cols = []
    order_by_cols = []
    limit_num = None
    
    for token in doc:
        if token.pos_ == "VERB":
            verb = token
            break
    
    for child in verb.children:
        if child.dep_ == "nsubj":
            # text = '_'.join(child.text).lower()
            text = child.text
            select_cols.append(text)
        elif child.dep_ == "dobj":
            # text = '_'.join(child.text).lower()
            text = child.text
            select_cols.append(text)
        elif child.dep_ == "prep":
            if child.text == "from":
                from_table = verb.doc[child.i + 1:].text
            elif child.text == "by":
                group_by_cols = [tok.text for tok in child.children if tok.pos_ == "NOUN"]
                order_by_cols = [tok.text for tok in child.children if tok.pos_ == "ADJ"]
                if not order_by_cols:
                    order_by_cols = group_by_cols
            elif child.text == "with":
                where_cond = verb.doc[child.i + 1:].text
            elif child.text == "limit":
                limit_num = int(child.nbor().text)
    
    query = "df"
    if where_cond:
        query += f"[{where_cond}]"
    query += f".sort_values(by={order_by_cols})" if order_by_cols else ""
    query += f".groupby(by={group_by_cols}).mean()" if group_by_cols else ""
    query += f"[{select_cols}]"
    query += f".head({limit_num})" if limit_num else ""

    print(query)
    
    return eval(query)

In [20]:
df = pd.DataFrame({
    "student_id": [1, 2, 3],
    "student_name": ["Alice", "Bob", "Charlie"],
    "student_grade": [85, 92, 78]
})

query = "Show student_name, and student_grade ordered by their grade"
result = process_query(query, df)
print(result)

df[['student_name']]
  student_name
0        Alice
1          Bob
2      Charlie


In [16]:
df = pd.DataFrame({
    "product_name": ["iPhone", "Samsung Galaxy", "Pixel", "OnePlus"],
    "manufacturer": ["Apple", "Samsung", "Google", "OnePlus"],
    "price": [999, 799, 699, 699],
    "release_date": ["2021-09-14", "2021-01-29", "2019-10-15", "2021-03-23"]
})

query = "Show the average price of phones released in 2021 made by Samsung and Google"
result = process_query(query, df)
print(result)

0    999
1    799
2    699
3    699
Name: price, dtype: int64
